In [221]:
# pip install networkx==2.8.8
# pip install matplotlib == 3.2.2

In [222]:
import networkx as nx
import dgl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

* session: 1~13
* round數量不一定

In [223]:
feature_df = pd.read_csv('data/graph_feature.csv')
structure_df = pd.read_csv('data/graph_structure.csv')

In [224]:
feature_df

,session,uid,round,size,color,score,num_of_neighbor,hist_color,hist_neighbor,hist_skip
0,1,1,1,15,4,0,6,1.00,0.00,0.00
1,1,2,1,15,4,0,6,1.00,0.00,0.00
2,1,3,1,15,4,0,6,1.00,0.00,0.00
3,1,4,1,15,4,0,6,1.00,0.00,0.00
4,1,5,1,15,4,0,6,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
2148,13,16,6,20,2,100,7,0.17,0.00,0.83
2149,13,17,6,20,3,100,7,0.17,0.17,0.67
2150,13,18,6,20,1,89,9,0.33,0.17,0.50
2151,13,19,6,20,4,100,7,0.00,0.50,0.50


In [225]:
structure_df

,session,uid,round,size,linked
0,1,1,1,15,"2,3,4,13,14,15"
1,1,2,1,15,"1,3,4,5,14,15"
2,1,3,1,15,"1,2,4,5,6,15"
3,1,4,1,15,"1,2,3,5,6,7"
4,1,5,1,15,"2,3,4,6,7,8"
...,...,...,...,...,...
2148,13,16,6,20,"13, 14, 15, 17, 18, 8, 19"
2149,13,17,6,20,"14, 15, 16, 18, 19, 4, 6"
2150,13,18,6,20,"1, 15, 16, 17, 19, 20, 7, 5, 11"
2151,13,19,6,20,"2, 17, 18, 4, 16, 5, 6"


用structure建立graph

In [226]:
# session_size_dict = {}
# for i in structure_df.drop_duplicates('session')[['session', 'size']].iloc:
#     session_size_dict[i[0]] = i[1]

In [227]:
def show_graph(nx_g):
    plt.figure(figsize=[5,5])
    # nx.draw(nx_g, with_labels=True)
    # plt.show()
    labels = {}
    for i in range(1, len(nx_g.nodes())+1):
        labels[i] = i
    pos = nx.circular_layout(nx_g)  # 環狀布圖
    nx.draw_networkx_nodes(nx_g, pos, node_color='tab:blue')
    nx.draw_networkx_labels(nx_g, pos, labels, font_size=10, font_color='black')
    nx.draw_networkx_edges(nx_g, pos, alpha=0.5, width=1)

In [228]:
def create_graph(session, df):  # df size = graph size
    size = len(df)+1
    G = nx.Graph()
    nodes = range(1, size)
    G.add_nodes_from(nodes)
    edge_list = []
    for uid in range(1, size):
        neighbor_list = df[df['uid'] == uid].iloc[0]['linked'].split(',')
        edge_list = [(int(uid), int(neighbor)) for neighbor in neighbor_list]
        G.add_edges_from(edge_list)
    return G

def get_training_data(session, round):
    test_df = structure_df[structure_df['session'] == session]
    test_df = test_df[test_df['round'] == round]

    graph = create_graph(1, test_df)
    show_graph(graph)
    return graph

feature: color, score, num_of_neighbor, hist_color, hist_neighbor, hist_skip

In [229]:
def add_feature(graph, f_df):
    feature_col_list = ['color', 'score', 'num_of_neighbor', 'hist_color', 'hist_neighbor', 'hist_skip']
    for n in graph.nodes():
        for f in feature_col_list:
            graph.nodes[n][f] = f_df[f_df['uid']==n][f].tolist()[0]    # 取出uid符合的user的f欄位，從series型態轉成list再取值
    return graph

In [239]:
graph_list = []
for session in range(1, 14):
    s_df = structure_df[structure_df['session'] == session]
    for round in s_df['round'].unique():
        s_df = s_df[s_df['round'] == round]
        f_df = feature_df.iloc[s_df.index]
        graph = create_graph(session, s_df)
        graph = add_feature(graph, f_df)
        graph_list.append(graph)

In [240]:
len(graph_list)

106